# Algorithms behind Multi-Model Recommender

## Architecture of Prediction.io

![Alt](files/pio-architecture.png "Architecture")
<center><b><u>Prediction.io Architecture</u></b></center>

<p/>
## Demo

In [163]:
import numpy as np
import pandas
from IPython.display import display, HTML
from tabulate import tabulate

def displayTable(table, tableType, title = None):
    displayTables([table], [tableType], [title])
    
def displayTables(tables, tableTypes, titles = None, width = 60):
    if titles is None:
        titles = [None for i in range(len(tables))]
        
    html = '<table id="table123" border="0" cellpadding="10", width="' + \
                    str(width) + '%", align="center"><tr>'
    for table, tableType, title in zip(tables, tableTypes, titles):
        if tableType == "p": # Purchase history
            if title is None: title = "Purchase History:"
            row_labels = ['u'+str(i+1) for i in range(table.shape[0])]
            col_labels = ['t'+str(i+1) for i in range(table.shape[1])]

        elif tableType == 'c': # Co-occurrence matrix
            if title is None: title = "Co-occurrence:"
            row_labels = ['t'+str(i+1) for i in range(table.shape[1])]
            col_labels = row_labels

        elif tableType == 'cg': # Co-occurrence matrix with gender
            if title is None: title = "Co-occurrence with Gender:"
            row_labels = ['M', 'F']
            col_labels = ['t'+str(i+1) for i in range(table.shape[1])]

        elif tableType == 'v': # Just a vector
            if table.shape[0] > 1:
                table = table.transpose()
            row_labels = ['user']
            col_labels = ['t'+str(i+1) for i in range(table.shape[1])]
         
        elif tableType == 'g': # Gender info
            if title is None: title = "Gender:"
            row_labels = ['u'+str(i+1) for i in range(table.shape[0])]
            col_labels = ['M', 'F']

        html += "<td>"
        if title is not None:
            html += "<b>" + title + "</b><br/>"

        df = pandas.DataFrame(table, columns=col_labels, index=row_labels)
        html += df.to_html()
        
        html += "</td>"
    
    html += "</tr></table>"
    display(HTML(html))

## Purchase History Matrix and Co-Occurrence Matrix
Assume we have 5 users and 6 products.
- Matrix A will be the purchase history matrix
- A'A is the co-occurence matrix.
    - Each value represents the number of times two items co-occur

Examples:
- item 1 and item 3 co-occur twice (in user1 and user2)
- item 2 and item 4 co-occur twice (in user3 and user5)

In [144]:
A = np.matrix([[1,0,1,0,0,0], [1,0,1,0,1,0], [0,1,1,1,0,0], [0,1,0,1,0,1], [0,0,0,0,0,0]]) # History of purchase

AtA = A.transpose() * A
np.fill_diagonal(AtA, 0) # Fill the diagnonal with zeros, as they should be ignored.

displayTables([A, AtA], ['p', 'c'])


,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
t1,0,0,2,0,1,0
t2,0,0,1,2,0,1
t3,2,1,0,1,1,0
t4,0,2,1,0,0,1


**Simple user-centric recommendation**

If a user is viewing item 1, what should we recommend?

In [106]:
h = np.matrix([1,0,0,0,0,0])

r = h * AtA  # Note: AtA * h' will produce the same result
displayTable(r, 'v')
# It's the same as reading the 1st row of AtA

,t1,t2,t3,t4,t5,t6
user,0,0,2,0,1,0


The above result tells us that we should recommend item 3, and then item 5.  It's based on the co-occurrence matrix:
- Two people who purchased item 1 also purchased item 3.
- One person who purchased item 1 also purchased item 5

This is similar to the "people who purchased this also purchase" list.

Now let's say a user comes in, and in her history she has purchased item 1 and item 2 in the past.  What should we recommend?

In [145]:
p = np.matrix([1,1,0,0,0,0])
displayTables([A, p * AtA], ['p', 'v'], 
              [None, "History: t1, t2<br/>Recommendations = p * A'A:"])
# It's the same as adding up the first two rows of AtA

,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
user,0,0,3,2,1,1


##Cross Co-occurrence##

In addition to purchase history, we also have Bookmark history of the 6 products.

**Cross-Cooccurrence Matrix**

In B'A, each row represent a bookmark, and each column represent a purchase.

In [146]:
B = np.matrix([[0,0,0,1,0,0], [0,0,0,1,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0], [1,0,0,0,1,1]]) # History of bookmark

BtA = B.transpose() * A

displayTables([A, B, BtA], ['p', 'p', 'c'], [None, 'Bookmark History:', "B'A:" ], width = 75)

,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
u1,0,0,0,1,0,0
u2,0,0,0,1,0,0
u3,0,1,0,0,0,0
u4,0,1,0,0,0,0


*Recommend a purchase based on bookmark history*

In [149]:
displayTables([A, B], ['p', 'p'], [None, 'Bookmark History:'], width=50)

print
print "Bookmark: 4   -", np.matrix([0,0,0,1,0,0]) * BtA
print "Bookmark: 1   -", np.matrix([1,0,0,0,0,0]) * BtA
print "Bookmark: 1&2 -", np.matrix([1,1,0,0,0,0]) * BtA

,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
u1,0,0,0,1,0,0
u2,0,0,0,1,0,0
u3,0,1,0,0,0,0
u4,0,1,0,0,0,0



Bookmark: 4   - [[2 0 2 0 1 0]]
Bookmark: 1   - [[0 0 0 0 0 0]]
Bookmark: 1&2 - [[0 2 1 2 0 1]]


Please note that in the last two cases, only user 5 have bookmarked items 1. But since he hasn't purchased any item, so we **cannot** correlate those this bookmark to any purchase. So the bookmark has **no** effect on the recommendation. 

### Recommend a purchase based on both purchase history and bookmark history

In [151]:
displayTables([A, B], ['p', 'p'], [None, 'Bookmark History:'])

p = np.matrix([1,0,1,0,0,0])
b = np.matrix([0,1,0,0,0,0])
r = p*AtA + b*BtA

print
print "A user's purchase:", p
print "Recommendations:  ", p*AtA, "\n"
print "A user's bookmark:", b
print "Recommendations:  ", b*BtA, "\n"
print "Combined:         ", p*AtA + b*BtA, "\n"


,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
u1,0,0,0,1,0,0
u2,0,0,0,1,0,0
u3,0,1,0,0,0,0
u4,0,1,0,0,0,0



A user's purchase: [[1 0 1 0 0 0]]
Recommendations:   [[2 1 2 1 2 0]] 

A user's bookmark: [[0 1 0 0 0 0]]
Recommendations:   [[0 2 1 2 0 1]] 

Combined:          [[2 3 3 3 2 1]] 



### What if we have users' gender information?

Let's say we have a new matrix, G, which record the gender information.  The columns represent Male and Female.

In [153]:
G = np.matrix([[1,0], [0,1], [0,1], [1,0], [0,1]])
displayTables([A, G], ['p', 'g'], width=40)

*For a female customer, what should we recommend?*

In [156]:
GtA = G.transpose() * A
displayTable(GtA, 'cg', "G'A:")

g = np.matrix([0,1])
print "For a female, we recommend: ", g * GtA

,t1,t2,t3,t4,t5,t6
M,1,1,1,1,0,1
F,1,1,2,1,1,0


For a female, we recommend:  [[1 1 2 1 1 0]]


### Recommend based on purchase history, bookmark history and gender info

In [160]:
displayTables([A, B, GtA], ['p', 'p', 'cg'], [None, "Bookmark History:", None], width=75)

print "A user's purchase:", p
print "Recommendations:  ", p*AtA, "\n"
print "A user's bookmark:", b
print "Recommendations:  ", b*BtA, "\n"
print "A user's gender:  ", g
print "Recommendations:  ", g*GtA, "\n"

print "Combined (two):   ", p*AtA + b*BtA, "\n"
print "Combined (all):   ", p*AtA + b*BtA + g*GtA, "\n"


,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
u1,0,0,0,1,0,0
u2,0,0,0,1,0,0
u3,0,1,0,0,0,0
u4,0,1,0,0,0,0


A user's purchase: [[1 0 1 0 0 0]]
Recommendations:   [[2 1 2 1 2 0]] 

A user's bookmark: [[0 1 0 0 0 0]]
Recommendations:   [[0 2 1 2 0 1]] 

A user's gender:   [[0 1]]
Recommendations:   [[1 1 2 1 1 0]] 

Combined (two):    [[2 3 3 3 2 1]] 

Combined (all):    [[3 4 5 4 3 1]] 



## Similar Items



In [161]:
displayTables([A,AtA], ['p', 'c'])

,t1,t2,t3,t4,t5,t6
u1,1,0,1,0,0,0
u2,1,0,1,0,1,0
u3,0,1,1,1,0,0
u4,0,1,0,1,0,1
u5,0,0,0,0,0,0
,t1,t2,t3,t4,t5,t6
t1,0,0,2,0,1,0
t2,0,0,1,2,0,1
t3,2,1,0,1,1,0
t4,0,2,1,0,0,1


Take item 4 as an example.  

Its row in AtA is `[0 2 1 0 0 1]`.  So it occurs the most with item 2, and also with 3 and 6.  So items 2, 3 and 6 are similar to it (based on purchase history).  

We can obtain the same result by multiplying AtA with an item vector.

In [162]:
t = np.matrix([0,0,0,1,0,0])
t * AtA

matrix([[0, 2, 1, 0, 0, 1]])

So t * AtA produces the "similar item" vector based on just the purchase history.


## Summary \#1
- Purchase is our main event.
- Matrix **A** contains the history of all user purchases.
- We have have secondary "events": **B** - Bookmark, and **G** - Gender 
- Let **a**, **b** and **g** be *vectors* representing the purchase history, current bookmarks and gender of a user
- Recommendation:  `r = a A'A + b B'A + g G'A + ...`

### Sidetrack 1: Log-Likelihood Ratio (LLR) Analysis

A key characteristic of a successful recommender engine is serendipty.

**Serendipity** - Serendipity is a measure "how surprising the recommendations are". For instance, a recommender system that recommends milk to a customer in a grocery store, might be perfectly accurate but still it is not a good recommendation because it is an obvious item for the customer to buy.

In our co-occurrence matrix, we want to filter out pairs which are *not* surprising.


In [203]:
def printSubTotalTable(table, headers, rows):
    table[2,0] = np.sum(table[:,0]) 
    table[2,1] = np.sum(table[:,1]) 
    table[0,2] = np.sum(table[0,:])
    table[1,2] = np.sum(table[1,:])
    
    tableNew = np.concatenate((np.array(rows).reshape(-1,1), table), axis=1)
    tableNew[2,3] = ''
    print tabulate(tableNew, headers, tablefmt='grid')
    print "%% of cake-eater who drinks coffee = %d/%d = %.2f%%" % \
        (table[0,0], table[2,0], (table[0,0]*1.0/table[2,0]*100))
    print "%% of population who drinks coffee = %d/%d = %.2f%%" % \
        (table[0,2], (table[0,2]+table[1,2]), (table[0,2]*1.0/(table[0,2]+table[1,2])*100))
    
headers = ['', 'Cake', 'Others', '']
rows = ['Coffee', 'Others', '']

case1 = np.array(
    [
        [13, 1000, 0],
        [1000, 100000, 0],
        [0, 0, 0]
    ])

print "Case 1"
printSubTotalTable(case1, headers, rows)

print
print "Case 2"
case2 = np.array(
    [
        [10, 3, 0],
        [2, 100000, 0],
        [0, 0, 0]
    ])
printSubTotalTable(case2, headers, rows)


Case 1
+--------+--------+----------+--------+
|        |   Cake |   Others |        |
+========+========+==========+========+
| Coffee |     13 |     1000 | 1013   |
+--------+--------+----------+--------+
| Others |   1000 |   100000 | 101000 |
+--------+--------+----------+--------+
|        |   1013 |   101000 |        |
+--------+--------+----------+--------+
% of cake-eater who drinks coffee = 13/1013 = 1.28%
% of population who drinks coffee = 1013/102013 = 0.99%

Case 2
+--------+--------+----------+--------+
|        |   Cake |   Others |        |
+========+========+==========+========+
| Coffee |     10 |        3 | 13     |
+--------+--------+----------+--------+
| Others |      2 |   100000 | 100002 |
+--------+--------+----------+--------+
|        |     12 |   100003 |        |
+--------+--------+----------+--------+
% of cake-eater who drinks coffee = 10/12 = 83.33%
% of population who drinks coffee = 13/100015 = 0.01%


The engine uses a test called *Log-Likelihood Ratio Analysis (LLR)* to **filter** out all non-interesting co-occurrence:<br/>
- All non-interesting cases become zero
- All interesting cases become one

So our co-occurrence matrix A'A may become like this:

In [209]:
AtA_llr = np.array([[0,0,0,0,0,1],[0,0,1,1,0,0],[0,1,0,0,0,0],[0,1,0,0,0,1],[1,0,0,0,0,0],[1,0,0,1,0,0]])
displayTables([AtA, AtA_llr], ['c','c'], ["A'A", "LLR(A'A)"])

,t1,t2,t3,t4,t5,t6
t1,0,0,2,0,1,0
t2,0,0,1,2,0,1
t3,2,1,0,1,1,0
t4,0,2,1,0,0,1
t5,1,0,1,0,0,0
t6,0,1,0,1,0,0
,t1,t2,t3,t4,t5,t6
t1,0,0,0,0,0,1
t2,0,0,1,1,0,0
t3,0,1,0,0,0,0


**The recommendation equation becomes:**<br/>
`r = a LLR(A'A) + b LLR(B'A) + g LLR(G'A)`

### Sidetrack 2: kNN using [cosine distance][1] and Search Engine
[1]: http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.spatial.distance.cosine.html "Cosine Distance"

In [219]:
p = np.matrix([0,0,1,0,1,0])
displayTables([AtA, p, p*AtA], ['c', 'v', 'v'], \
              [None, "A user's purchase history", "Recommendation = p A'A"], width=90)

print
from scipy.spatial import distance
for r in range(6):
    dist = distance.cosine(p, AtA[r])
    print "Item", r + 1, "- cosine distance with p =", ("-" if p[0,r] == 1 else dist)

,t1,t2,t3,t4,t5,t6
t1,0,0,2,0,1,0
t2,0,0,1,2,0,1
t3,2,1,0,1,1,0
t4,0,2,1,0,0,1
t5,1,0,1,0,0,0
t6,0,1,0,1,0,0
,t1,t2,t3,t4,t5,t6
user,0,0,1,0,1,0
,t1,t2,t3,t4,t5,t6
user,3,1,1,1,1,0



Item 1 - cosine distance with p = 0.0513167019495
Item 2 - cosine distance with p = 0.711324865405
Item 3 - cosine distance with p = -
Item 4 - cosine distance with p = 0.711324865405
Item 5 - cosine distance with p = -
Item 6 - cosine distance with p = 1.0


Based on nearest neighbour, the three closest items are 1, 2, and 4.  That matches our earlier results.

**Intuition:** <br/>
- In AtA, t1 co-occurs with t3 and t5.  
- If our history (p) contains both t3 and t5, the cosine distance between p and AtA[0] is short, and it means we should buy t1.


## Search Engine as prediction engine
Remember our recommendation formula:
`r = a LLR(A'A) + b LLR(B'A) + g LLR(G'A) + ...`

The Universal Recommender actually stores `LLR(A'A), LLR(B'A) and LLR(G'A)` as three searcheable fields in Elastic Search:
- For each item UR stores its corresponding rows from the indicator matrices in the search engine index.
- Item's properties (e.g. categories, price) are also stored as fields in search engine.
- Using search engine we have lots of flexibilities in boosting and filtering.

** Demo **

## Summary \#2
- Purchase is our main event.
- Matrix A contains the history of all user purchases.
- We have have secondary "events": B - Bookmark, and G - Gender, **and more....**
- Let a, b and g be *vectors* representing the purchase history, current bookmarks and gender of a user
- Recommendation:  `r = a LLR(A'A) + b LLR(B'A) + g LLR(G'A) + ...`
- We can define properties for our item:
    - Price
    - Brand
    - etc.
- Using Search Engine, we can perform filtering and boosting.  E.g.:
    - Boost a certain brand
    - Boost a certain price range
    - Limit to a certain price range
    
## Evaluation
- Pros
    - Open Source - can study and modify the source code.
    - The algorithm was suggested by Ted Dunning
        - Chief Application Architect at MapR
        - Ted was the chief architect behind the MusicMatch (now Yahoo Music) and Veoh recommendation systems.
- Cons
    - The Universal Recommender is relatively new - v0.2 released in July, now at v0.2.2
    - Primary event: need to choose a good subset of events from Price.com
    - Not enough data for testing.  Need to wait for more.